# Typing Changes

> Note: Here we use typing-extensions module instead of typing, as tools like pylance were not updated at the time.

### typing.Never
Never type is a "bottom type" meaning matches nothing. This is the opposite of `typing.Any` which matches all types. 

#### Never as NoReturn
Never as a return type is equivalent to `typing.NoReturn`.

In [5]:
from typing_extensions import Never


def alway_raise() -> Never:  # Same as NoReturn
    raise Exception("blah")


def sometimes_raise(error: bool) -> Never:
    if not error:
        return  # type checker ⭕
    raise Exception("blah")

#### Never as an argument

Never as an argument is not really something that'll be used frequently in day-to-day development, it's instead used to provide a method to type check unreachable code.

In [4]:
from typing_extensions import Never


def never_called(value: Never) -> None:
    pass


def main(a: int):
    if isinstance(a, int):
        never_called(a)  # type checker ⭕
        return

    never_called(a)  # type checker 💚

    # NOTE: Could also have done assert, but runtime behaviour
    assert False


There is actually a convenient way `typing.assert_never` method for this.

In [17]:
from typing_extensions import Never, assert_never, assert_type, reveal_type


def identify(a: int | str) -> str:
    if isinstance(a, int):
        return "it's an int"

    # type narrowing: a could only be a string here
    assert_type(a, str)
    if isinstance(a, str):
        return "it's a string"

    # further narrowed to Never here
    reveal_type(a)
    assert_type(a, Never)  # runtime this is a no-op
    assert_never(a)  # Same as above but also fails at run time


Let's check the runtime behaviour of the 3 new functions:

In [22]:
identify(None)

Runtime type is 'NoneType'


AssertionError: Expected code to be unreachable, but got: None

- typing.reveal_type: Forces the static type checker to output the type, and prints the type at runtime.
- typing.assert_type: Forces the static type checker to check the type, no effect at runtime.
- typing.assert_never: Same as `assert_type(blah, Never)` but also gives a runtime assertion error

#### PEP 646: Variadic generics

First let's review generics:

In [6]:
from typing import Generic, TypeVar

T = TypeVar("T")


class MyList(Generic[T]):
    def __getitem__(self, index: int) -> T:
        ...


T_KEY = TypeVar("T_KEY")
T_VAL = TypeVar("T_VAL")


class MyDict(Generic[T_KEY, T_VAL]):
    def __getitem__(self, key: T_KEY) -> T_VAL:
        ...

The tuple type allows for a variable number of type variables, for example:

In [11]:
a: tuple[int, int] = (1, 2)
b: tuple[str, ...] = ('', '', '')

We can define similar generics using `typing.TypeVarTuple`

In [7]:
from typing import Generic
from typing_extensions import TypeVarTuple, Unpack

KEYS = TypeVarTuple("KEYS")


class Row(Generic[Unpack[KEYS]]):
    @property
    def pk(self) -> tuple[Unpack[KEYS]]:
        ...
    

class User(Row[str]):
    ...


class Item(Row[int]):
    ...


class UserItem(Row[str, int]):
    ...


In 3.11 you can use a * expression instead of Unpack.

```python
class Row(Generic[*KEYS]):
    ...
```

`TypeVarTuple` could also be used similar to `ParamSpec` to define change in a function's parameter.

In [8]:
from typing import Callable
from typing_extensions import TypeVarTuple, Unpack

P = TypeVarTuple("P")
T = TypeVar("T")


def add_context(fn: Callable[[Unpack[P], dict], T]) -> Callable[[Unpack[P]], T]:
    def _new_fn(*args: Unpack[P]):
        return fn(*args, {})

    return _new_fn


@add_context
def operation(a: int, b: int, context: dict) -> int:
    return a + b


operation(1, 2)


3

### PEP 673: Self type

In [9]:
class Base:
    @classmethod
    def from_str(cls, input_string: str) -> "Base":
        return cls()


class Derived(Base):
    pass


d: Derived = Derived.from_str("my_string")  # type checker: ⭕

We can attempt to fix it like so:

In [10]:
from typing import TypeVar

T = TypeVar("T")


class Base2:
    @classmethod
    def from_str(cls: type[T], input_string: str) -> "T":
        return cls()


class Derived2(Base2):
    pass


d2: Derived2 = Derived2.from_str("my_string")  # type checker: 💚


Self can make this a lot clearer:

In [11]:
from typing_extensions import Self

class Base3:
    @classmethod
    def from_str(cls, input_string: str) -> Self:
        return cls()


class Derived3(Base3):
    pass


d3: Derived3 = Derived3.from_str("my_string")  # type checker: 💚

### PEP 675: Arbitrary literal string type

In [12]:
from typing_extensions import LiteralString

def execute(sql_query: LiteralString):
    return None


execute("SELECT * FROM table")  # type checker: 💚
execute("SELECT * FROM table" + "WHERE a = 2")  # type checker: 💚

where_clause: LiteralString = "WHERE a = 2"

execute(f"SELECT * FROM table {where_clause}")  # type checker: ⭕
execute("SELECT * FROM table {}".format(where_clause))  # type checker: ⭕


def generate_where_clause() -> str:
    return "INjection"


execute("SELECT * FROM table" + generate_where_clause())  # type checker: ⭕


### PEP 655: Marking individual TypedDict items as required or not-required

In [13]:
from typing import TypedDict
from typing_extensions import NotRequired, Required


class MongoDoc(TypedDict):
    _id: str
    data: NotRequired[list]


class MongoDocV2(TypedDict, total=False):
    _id: Required[str]
    data: list


m1: MongoDoc = {"_id": "abcd"}
m1 = {"_id": "abcd", "something_else": None}  # type checker: ⭕
m2: MongoDocV2 = {"_id": "abcd"}
m2 = {"_id": "abcd", "something_else": None}  # type checker: ⭕


### typing.Any Changes

In [14]:
class FakeEvent:
    pass


class Event:
    pass


def print_event(event: Event) -> None:
    print(f"The event is {event}")


print_event(FakeEvent)  # type checker ⭕

The event is <class '__main__.FakeEvent'>


Instead we can subclass `Any`, since an input of `Any` type will bind to any function argument.

In [15]:
from typing import Any

class FakeEvent2(Any):
    ...


print_event(FakeEvent2())

The event is <__main__.FakeEvent2 object at 0x7f603186fc50>


#### PEP 681: typing.dataclass_transform

dataclass_transform allows 3rd party libraries to create their own "dataclass" wrappers and for type checkers to understand them. 

In [1]:
from typing import dataclass_transform


@dataclass_transform(kw_only_default=True)
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # Something like this


class Event(Record):
    event_id: int


Event(event_id=1)
